In [3]:
from scipy.ndimage.interpolation import shift
import numpy as np
import numpy
import numba

@numba.jit
def MFI_inc(high: numpy.ndarray, low: numpy.ndarray, close: numpy.ndarray, vol: numpy.ndarray, n: int):
    '''
    计算MFI标量
    需要n+1个HLCV
    '''
    assert len(high) == n+1
    typicalPrice = (high + low + close) / 3
    preTypicalPrice = shift(typicalPrice, 1, cval=np.nan)
    moneyFlow = typicalPrice * vol
    pmf = moneyFlow[typicalPrice > preTypicalPrice]
    pmfs = pmf.sum()
    mfs = moneyFlow[1:].sum()
    mfi = 100*pmfs/mfs
    
    return mfi
    
@numba.jit
def MFI(inputs: dict, n: int):
    '''
    计算MFI向量
    '''
    high = inputs['high']
    low = inputs['low']
    close = inputs['close']
    vol = inputs['volume']
    
    mfi = np.full_like(high, fill_value=np.nan)
    
    i = 0
    while i+n < len(high):
        mfi[i+n] = MFI_inc(high[i:i+n+1], low[i:i+n+1], close[i:i+n+1], vol[i:i+n+1], n)
        i += 1
        
    return mfi
    

In [5]:
inputs = {
    'open': np.random.random(100),
    'high': np.random.random(100),
    'low': np.random.random(100),
    'close': np.random.random(100),
    'volume': np.random.random(100)
}

In [76]:
MFI(inputs, 8) - talib.MFI()

<ipython-input-74-803842d33fca>:23: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "MFI" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-74-803842d33fca> (28)

File "<ipython-input-74-803842d33fca>", line 28:
def MFI(inputs: dict, n: int):
    <source elided>
    '''
    high = inputs['high']
    ^

  @numba.jit
<ipython-input-74-803842d33fca>:23: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "MFI" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-74-803842d33fca>", line 35:
def MFI(inputs: dict, n: int):
    <source elided>
    
    i = 0
    ^

  @numba.jit
/opt/anaconda3/lib/python3.6/site-packages/numba/compiler.py:742: NumbaWarning: Function "MFI" was compiled in object mode without forceobj=True, but has lifted loops.

File "<ipython-input-74-803

7.33 ms ± 58.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


In [6]:
import talib.abstract

talib.abstract.MFI(inputs, 8) - MFI(inputs, 8)

<ipython-input-3-803842d33fca>:23: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "MFI" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-3-803842d33fca> (28)

File "<ipython-input-3-803842d33fca>", line 28:
def MFI(inputs: dict, n: int):
    <source elided>
    '''
    high = inputs['high']
    ^

  @numba.jit
<ipython-input-3-803842d33fca>:23: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "MFI" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-3-803842d33fca>", line 35:
def MFI(inputs: dict, n: int):
    <source elided>
    
    i = 0
    ^

  @numba.jit
/opt/anaconda3/lib/python3.7/site-packages/numba/compiler.py:742: NumbaWarning: Function "MFI" was compiled in object mode without forceobj=True, but has lifted loops.

File "<ipython-input-3-803842d33

array([            nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -2.13162821e-14,  0.00000000e+00,  0.00000000e+00,
        1.42108547e-14,  0.00000000e+00,  0.00000000e+00, -1.42108547e-14,
        0.00000000e+00,  1.42108547e-14,  1.42108547e-14,  1.42108547e-14,
        7.10542736e-15,  2.13162821e-14, -7.10542736e-15,  0.00000000e+00,
        0.00000000e+00,  7.10542736e-15, -1.42108547e-14,  1.42108547e-14,
        0.00000000e+00,  1.42108547e-14,  7.10542736e-15,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  7.10542736e-15,  0.00000000e+00,
        0.00000000e+00, -7.10542736e-15, -7.10542736e-15, -7.10542736e-15,
        0.00000000e+00,  7.10542736e-15,  7.10542736e-15, -1.42108547e-14,
        0.00000000e+00,  1.42108547e-14,  2.84217094e-14,  0.00000000e+00,
        2.84217094e-14,  